In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel
from utils import extract_solution

![alternatvie text](pics/2023-07-07-11-43-28.png)

https://developers.google.com/optimization/cp/queens

In [2]:
mdl = CpoModel(name="NQueens")

In [3]:
board_size = 8  # boardsize

# Every column must have a queen, we set the row number as DV
queens = mdl.integer_var_list(min = 0, max =board_size-1, size = board_size, name = "x" )
queens = pd.DataFrame(queens, columns = ["queen_row_number"])
queens['queen_column_number']=queens.index
queens

,queen_row_number,queen_column_number
0,x_0 = intVar(0..7),0
1,x_1 = intVar(0..7),1
2,x_2 = intVar(0..7),2
3,x_3 = intVar(0..7),3
4,x_4 = intVar(0..7),4
5,x_5 = intVar(0..7),5
6,x_6 = intVar(0..7),6
7,x_7 = intVar(0..7),7


In [4]:
# The row numbers of queens must be different

mdl.add( mdl.all_diff(queens['queen_row_number']) )

The diagonal constraint is a little trickier than the row and column constraints. First, if two queens lie on the same diagonal, one of the following conditions must be true:

The row number plus the column number for each of the two queens are equal. In other words, $queens(j) + j$ has the same value for two different indices $j$.

The row number minus the column number for each of the two queens are equal. In this case, $queens(j) - j$ has the same value for two different indices $j$.

One of these conditions means the queens lie on the same ascending diagonal ( going from left to right), while the other means they lie on the same descending diagonal. Which condition corresponds to ascending and which to descending depends on how you order the rows and columns.

So the diagonal constraint is that the values of $queens(j) + j$ must all be different, and the values of $queens(j) - j$ must all be different, for different $j$.

In [5]:
queens['U']= queens.apply(lambda x:x['queen_row_number']+x['queen_column_number'], axis=1)
queens['V']= queens.apply(lambda x:x['queen_row_number']-x['queen_column_number'], axis=1)

display(queens)

mdl.add( mdl.all_diff(queens['U']) )
mdl.add( mdl.all_diff(queens['V']) )

,queen_row_number,queen_column_number,U,V
0,x_0 = intVar(0..7),0,x_0 + 0,x_0 - 0
1,x_1 = intVar(0..7),1,x_1 + 1,x_1 - 1
2,x_2 = intVar(0..7),2,x_2 + 2,x_2 - 2
3,x_3 = intVar(0..7),3,x_3 + 3,x_3 - 3
4,x_4 = intVar(0..7),4,x_4 + 4,x_4 - 4
5,x_5 = intVar(0..7),5,x_5 + 5,x_5 - 5
6,x_6 = intVar(0..7),6,x_6 + 6,x_6 - 6
7,x_7 = intVar(0..7),7,x_7 + 7,x_7 - 7


In [6]:
msol = mdl.solve(TimeLimit=10, log_output = None)
#msol.print_solution()

In [7]:
types = [type(i) for i in queens.iloc[0].values]
queens_solution = extract_solution(queens,mdl, msol, drop = False )

queens_solution

,queen_row_number,queen_column_number,U,V,queen_row_number_Solution,U_Solution,V_Solution
0,x_0 = intVar(0..7),0,x_0 + 0,x_0 - 0,4,4,4
1,x_1 = intVar(0..7),1,x_1 + 1,x_1 - 1,1,2,0
2,x_2 = intVar(0..7),2,x_2 + 2,x_2 - 2,7,9,5
3,x_3 = intVar(0..7),3,x_3 + 3,x_3 - 3,0,3,-3
4,x_4 = intVar(0..7),4,x_4 + 4,x_4 - 4,3,7,-1
5,x_5 = intVar(0..7),5,x_5 + 5,x_5 - 5,6,11,1
6,x_6 = intVar(0..7),6,x_6 + 6,x_6 - 6,2,8,-4
7,x_7 = intVar(0..7),7,x_7 + 7,x_7 - 7,5,12,-2


In [8]:
queens_solution['queen_row_number_Solution'].values

array([4, 1, 7, 0, 3, 6, 2, 5], dtype=int64)

In [9]:
for i in range(0,board_size):
    for j in range(0,board_size):
            m = queens_solution[(queens_solution['queen_row_number_Solution']==i) & (queens_solution['queen_column_number']==j)  ]
            if len(m) ==0:
                  print('_', end=' ')
            else:
                  print('Q', end=' ')
    print()
print()

_ _ _ Q _ _ _ _ 
_ Q _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ 
Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ Q 
_ _ _ _ _ Q _ _ 
_ _ Q _ _ _ _ _ 



In [10]:
from docplex.cp.utils import CpoNotSupportedException

siter = mdl.start_search(SearchType='DepthFirst', Workers=1, TimeLimit=100, log_output = None) # Parameters needed to avoid duplicate solutions
try:
    print("Row numbers for queens in columns 1 to board_size")
    for i, msol in enumerate(siter):
        print("Solution_"+str(i + 1) + ": ")
        queens_solution = extract_solution(queens,mdl, msol, drop = False )
        print(queens_solution['queen_row_number_Solution'].values)
        
except CpoNotSupportedException:
    print("This instance of the solver does not support solution iteration.")

Row numbers for queens in columns 1 to board_size
Solution_1: 
[6 3 1 7 5 0 2 4]
Solution_2: 
[3 6 4 1 5 0 2 7]
Solution_3: 
[3 1 4 7 5 0 2 6]
Solution_4: 
[3 1 7 4 6 0 2 5]
Solution_5: 
[3 5 7 1 6 0 2 4]
Solution_6: 
[6 3 1 4 7 0 2 5]
Solution_7: 
[4 6 1 3 7 0 2 5]
Solution_8: 
[4 6 1 5 2 0 3 7]
Solution_9: 
[2 6 1 7 4 0 3 5]
Solution_10: 
[6 2 7 1 4 0 5 3]
Solution_11: 
[4 2 7 3 6 0 5 1]
Solution_12: 
[2 5 1 4 7 0 6 3]
Solution_13: 
[2 5 7 1 3 0 6 4]
Solution_14: 
[1 3 5 7 2 0 6 4]
Solution_15: 
[2 5 1 6 4 0 7 3]
Solution_16: 
[4 6 1 5 2 0 7 3]
Solution_17: 
[5 2 0 7 3 1 6 4]
Solution_18: 
[2 4 6 0 3 1 7 5]
Solution_19: 
[5 2 0 7 4 1 3 6]
Solution_20: 
[3 6 0 7 4 1 5 2]
Solution_21: 
[7 3 0 2 5 1 6 4]
Solution_22: 
[3 7 0 2 5 1 6 4]
Solution_23: 
[4 7 3 0 6 1 5 2]
Solution_24: 
[3 7 0 4 6 1 5 2]
Solution_25: 
[2 0 6 4 7 1 3 5]
Solution_26: 
[4 2 0 5 7 1 3 6]
Solution_27: 
[5 3 6 0 7 1 4 2]
Solution_28: 
[0 6 3 5 7 1 4 2]
Solution_29: 
[5 3 0 4 7 1 6 2]
Solution_30: 
[4 0 3 5 7 1 6 2]